In [1]:
import pandas as pd
import numpy as np
import requests
from tqdm import tqdm
import psycopg2
from psycopg2 import sql
from psycopg2.extensions import register_adapter, AsIs
psycopg2.extensions.register_adapter(np.int64, psycopg2._psycopg.AsIs)

In [2]:
df = pd.read_csv('peer_groups.csv', delimiter=';')
df.head()

,Unternehmen,Branche Detail,Branche aggregiert,Link zu Kennzahlen,stock_ticker,Unnamed: 5,daily_stock
0,1&1 Drillisch,Telekommunikation,Software / Tech,https://www.finanzen.net/aktien/drillisch-aktie,DRI.XETRA,DRI.STU,DRI.XETRA
1,AAR Corp.,Raumfahrt,Aviation,https://www.finanzen.net/aktien/aar-aktie,AIR.US,NaN,AIR.US
2,Activision,Gaming / E-Sport,Software / Tech,https://www.finanzen.net/aktien/activision_bli...,AIY.XETRA,ATVI.US,AIY.XETRA
3,ADIDAS,Sportartikelhersteller,Handel,https://www.finanzen.net/aktien/adidas-aktie,ADS.XETRA,ADDYY.US,ADS.XETRA
4,Adler Real Estate,Immobilien,Immobilien,https://www.finanzen.net/aktien/adler_real_est...,ADL.XETRA,NaN,ADL.XETRA


In [4]:
def create_api(ticker):
    return('https://eodhistoricaldata.com/api/fundamentals/{}?api_token=5ef9e1d3d885e2.21351768'.format(ticker))


ticker = {}
count = 1

for idx, row in tqdm(df.iterrows()):
    try:
        response = requests.get(create_api(row['stock_ticker']))
        data = response.json()
        print(data.keys())
    except:
        print('error')
    break

0it [00:03, ?it/s]

dict_keys(['General', 'Highlights', 'Valuation', 'SharesStats', 'Technicals', 'SplitsDividends', 'outstandingShares', 'Earnings', 'Financials'])


In [59]:
try:
    conn = psycopg2.connect(host="95.217.128.174", database="barbossa", user="maximus", password="maximus")
    cur = conn.cursor()

    for index, row in df.iterrows():
        cur.execute(
            sql.SQL("insert into balance_sheet_quaterly values (%s)"), [3])

    conn.commit()
    print('{} DONE!'.format('test'))

except(Exception, psycopg2.DatabaseError) as error:
        print('{} Datenabfrage fehlgeschlagen'.format('test'))
        print(error)

test DONE!


In [113]:
# def pump_to_db(data):
try:
    conn = psycopg2.connect(host="95.217.128.174", database="barbossa", user="maximus", password="maximus")
    cur = conn.cursor()

    name = data['General']['Name']
    code = data['General']['Code']
    exchange = data['General']['Exchange']
    currency = data['General']['CurrencyCode']

    cur.execute(
        sql.SQL("insert into highlights values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, \
                                                %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, \
                                                %s, %s, %s, %s, %s, %s, %s, %s)"), [
            
            name,
            code,
            exchange,
            currency,
            highlights['MarketCapitalization'].values[0],
            highlights['MarketCapitalizationMln'].values[0],
            highlights['EBITDA'].values[0],
            highlights['PERatio'].values[0],
            highlights['PEGRatio'].values[0],
            highlights['WallStreetTargetPrice'].values[0],
            highlights['BookValue'].values[0],
            highlights['DividendShare'].values[0],
            highlights['DividendYield'].values[0],
            highlights['EarningsShare'].values[0],
            highlights['EPSEstimateCurrentYear'].values[0],
            highlights['EPSEstimateNextYear'].values[0],
            highlights['EPSEstimateNextQuarter'].values[0],
            highlights['EPSEstimateCurrentQuarter'].values[0],
            highlights['MostRecentQuarter'].values[0],
            highlights['ProfitMargin'].values[0],
            highlights['OperatingMarginTTM'].values[0],
            highlights['ReturnOnAssetsTTM'].values[0],
            highlights['ReturnOnEquityTTM'].values[0],
            highlights['RevenueTTM'].values[0],
            highlights['QuarterlyRevenueGrowthYOY'].values[0],
            highlights['GrossProfitTTM'].values[0],
            highlights['DilutedEpsTTM'].values[0],
            highlights['QuarterlyEarningsGrowthYOY'].values[0]
        ])

    conn.commit()
    print('{} DONE!'.format('test'))

except(Exception, psycopg2.DatabaseError) as error:
        print('{} Datenabfrage fehlgeschlagen'.format('test'))
        print(error)

test DONE!


In [105]:
highlights['DilutedEpsTTM']

0    2.116
Name: DilutedEpsTTM, dtype: float64

In [67]:
pump_to_db(data)

test Datenabfrage fehlgeschlagen
can't adapt type 'Series'


In [ ]:
data['General']['Code']
data['General']['Name']
data['General']['Exchange']
data['General']['CurrencyCode']

In [53]:
data['General']['Name']

'1&1 Drillisch AG'

In [108]:
highlights['EPSEstimateCurrentYear']

0   -1
Name: EPSEstimateCurrentYear, dtype: int64

In [114]:
highlights = pd.DataFrame(data['Highlights'], index=[0])
highlights.fillna(0, inplace=True)
highlights.columns

Index(['MarketCapitalization', 'MarketCapitalizationMln', 'EBITDA', 'PERatio',
       'PEGRatio', 'WallStreetTargetPrice', 'BookValue', 'DividendShare',
       'DividendYield', 'EarningsShare', 'EPSEstimateCurrentYear',
       'EPSEstimateNextYear', 'EPSEstimateNextQuarter',
       'EPSEstimateCurrentQuarter', 'MostRecentQuarter', 'ProfitMargin',
       'OperatingMarginTTM', 'ReturnOnAssetsTTM', 'ReturnOnEquityTTM',
       'RevenueTTM', 'RevenuePerShareTTM', 'QuarterlyRevenueGrowthYOY',
       'GrossProfitTTM', 'DilutedEpsTTM', 'QuarterlyEarningsGrowthYOY'],
      dtype='object')

In [91]:
len(highlights.columns)

25

In [98]:
values = []
values.append(data['General']['Name'])
values.append(data['General']['Code'])
values.append(data['General']['Exchange'])
values.append(data['General']['CurrencyCode'])


for col in highlights.columns:
    if highlights[col].values[0] != None:
        values.append(highlights[col].values[0])
    else:
        values.append(0)

In [99]:
values

['1&1 Drillisch AG',
 'DRI',
 'XETRA',
 'EUR',
 4097412608,
 4097.4126,
 930145024,
 11.1651,
 0.67,
 69.47,
 26.831,
 0.05,
 0.0021,
 2.116,
 0,
 0,
 0,
 0,
 '2020-03-31',
 0.1004,
 0.2172,
 0.0854,
 0.082,
 3713061888,
 21.077,
 0.042,
 1177741000,
 2.116,
 0]

In [79]:
[highlights.iloc[:, 1].values[0] for i in range(0, len(highlights.columns))]

[4097.4126,
 4097.4126,
 4097.4126,
 4097.4126,
 4097.4126,
 4097.4126,
 4097.4126,
 4097.4126,
 4097.4126,
 4097.4126,
 4097.4126,
 4097.4126,
 4097.4126,
 4097.4126,
 4097.4126,
 4097.4126,
 4097.4126,
 4097.4126,
 4097.4126,
 4097.4126,
 4097.4126,
 4097.4126,
 4097.4126,
 4097.4126,
 4097.4126]

In [76]:
highlights.iloc[:, i].values[0]

0

In [60]:
for item in highlights.columns:
    print(item)

MarketCapitalization
MarketCapitalizationMln
EBITDA
PERatio
PEGRatio
WallStreetTargetPrice
BookValue
DividendShare
DividendYield
EarningsShare
EPSEstimateCurrentYear
EPSEstimateNextYear
EPSEstimateNextQuarter
EPSEstimateCurrentQuarter
MostRecentQuarter
ProfitMargin
OperatingMarginTTM
ReturnOnAssetsTTM
ReturnOnEquityTTM
RevenueTTM
RevenuePerShareTTM
QuarterlyRevenueGrowthYOY
GrossProfitTTM
DilutedEpsTTM
QuarterlyEarningsGrowthYOY


In [116]:
valuation = pd.DataFrame(data['Valuation'], index=[0])
valuation.fillna(0, inplace=True)
for item in valuation.columns:
    print(item)

TrailingPE
ForwardPE
PriceSalesTTM
PriceBookMRQ
EnterpriseValueRevenue
EnterpriseValueEbitda


In [117]:
sharesstats = pd.DataFrame(data['SharesStats'], index=[0])
sharesstats.fillna(0, inplace=True)
for item in sharesstats.columns:
    print(item)

SharesOutstanding
SharesFloat
PercentInsiders
PercentInstitutions
SharesShort
SharesShortPriorMonth
ShortRatio
ShortPercentOutstanding
ShortPercentFloat


In [118]:
technicals = pd.DataFrame(data['Technicals'], index=[0])
technicals.fillna(0, inplace=True)
for item in technicals.columns:
    print(item)

Beta
52WeekHigh
52WeekLow
50DayMA
200DayMA
SharesShort
SharesShortPriorMonth
ShortRatio
ShortPercent


In [127]:
bs_quater = pd.DataFrame(data['Financials']['Balance_Sheet']['quarterly']).T

In [155]:
bsq_values = []

bsq_values.append(data['General']['Name'])
bsq_values.append(data['General']['Code'])
bsq_values.append(data['General']['Exchange'])
bsq_values.append(data['General']['CurrencyCode'])

for idx, row in bs_quater.iterrows():
    
    for item in row.values:
        bsq_values.append(item)
    break

In [158]:
for idx, row in bs_quater.iterrows():

    bsq_values = []
    bsq_values.append(data['General']['Name'])
    bsq_values.append(data['General']['Code'])
    bsq_values.append(data['General']['Exchange'])
    bsq_values.append(data['General']['CurrencyCode'])

    for item in row.values:
        bsq_values.append(item)
    break

In [159]:
bsq_values

['1&1 Drillisch AG',
 'DRI',
 'XETRA',
 'EUR',
 '2020-03-31',
 None,
 'EUR',
 '6544218000.00',
 '1665008000.00',
 None,
 '19061000.00',
 '1814920000.00',
 '4729298000.00',
 '249233000.00',
 '28974000.00',
 '193891000.00',
 '2106255000.00',
 '1270502000.00',
 '2932943000.00',
 '425875000.00',
 '9734000.00',
 '544418000.00',
 None,
 None,
 None,
 '2429152000.00',
 '64247000.00',
 '1454368000.00',
 None,
 '131347000.00',
 '29285000.00',
 '242623000.00',
 None,
 '72544000.00',
 '286332000.00',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 '5089850000.00',
 None,
 '5089850000.00',
 None,
 None,
 None,
 '1270502000.00',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [119]:
for quater in data['Financials']['Balance_Sheet']['quarterly']:
    print(pd.DataFrame(data['Financials']['Balance_Sheet']['quarterly'][quater], index=[0]))
    break

         date filing_date currency_symbol    totalAssets intangibleAssets  \
0  2020-03-31        None             EUR  6544218000.00    1665008000.00   

  earningAssets otherCurrentAssets      totalLiab totalStockholderEquity  \
0          None        19061000.00  1814920000.00          4729298000.00   

  deferredLongTermLiab  ... nonCurrentLiabilitiesTotal negativeGoodwill  \
0         249233000.00  ...              1270502000.00             None   

  warrants preferredStockRedeemable capitalSurpluse  \
0     None                     None            None   

  liabilitiesAndStockholdersEquity cashAndShortTermInvestments  \
0                             None                        None   

  propertyPlantAndEquipmentGross accumulatedDepreciation  \
0                           None                    None   

  commonStockSharesOutstanding  
0                         None  

[1 rows x 57 columns]


In [41]:
for year in data['Financials']['Balance_Sheet']['yearly']:
    print(pd.DataFrame(data['Financials']['Balance_Sheet']['yearly'][year], index=[0]).columns)
    break

Index(['date', 'filing_date', 'currency_symbol', 'totalAssets',
       'intangibleAssets', 'earningAssets', 'otherCurrentAssets', 'totalLiab',
       'totalStockholderEquity', 'deferredLongTermLiab', 'otherCurrentLiab',
       'commonStock', 'retainedEarnings', 'otherLiab', 'goodWill',
       'otherAssets', 'cash', 'totalCurrentLiabilities', 'shortTermDebt',
       'shortLongTermDebt', 'shortLongTermDebtTotal', 'otherStockholderEquity',
       'propertyPlantEquipment', 'totalCurrentAssets', 'longTermInvestments',
       'netTangibleAssets', 'shortTermInvestments', 'netReceivables',
       'longTermDebt', 'inventory', 'accountsPayable', 'totalPermanentEquity',
       'noncontrollingInterestInConsolidatedEntity',
       'temporaryEquityRedeemableNoncontrollingInterests',
       'accumulatedOtherComprehensiveIncome', 'additionalPaidInCapital',
       'commonStockTotalEquity', 'preferredStockTotalEquity',
       'retainedEarningsTotalEquity', 'treasuryStock',
       'accumulatedAmortizatio

In [42]:
for quater in data['Financials']['Cash_Flow']['quarterly']:
    print(pd.DataFrame(data['Financials']['Cash_Flow']['quarterly'][quater], index=[0]).columns)
    break

Index(['date', 'filing_date', 'currency_symbol', 'investments',
       'changeToLiabilities', 'totalCashflowsFromInvestingActivities',
       'netBorrowings', 'totalCashFromFinancingActivities',
       'changeToOperatingActivities', 'netIncome', 'changeInCash',
       'totalCashFromOperatingActivities', 'depreciation',
       'otherCashflowsFromInvestingActivities', 'dividendsPaid',
       'changeToInventory', 'changeToAccountReceivables',
       'salePurchaseOfStock', 'otherCashflowsFromFinancingActivities',
       'changeToNetincome', 'capitalExpenditures', 'changeReceivables',
       'cashFlowsOtherOperating', 'exchangeRateChanges',
       'cashAndCashEquivalentsChanges'],
      dtype='object')


In [16]:
for quater in data['Financials']['Income_Statement']['quarterly']:
    cols = pd.DataFrame(data['Financials']['Income_Statement']['quarterly'][quater], index=[0]).columns
    break

In [17]:
for col in cols:
    print(col)

date
filing_date
currency_symbol
researchDevelopment
effectOfAccountingCharges
incomeBeforeTax
minorityInterest
netIncome
sellingGeneralAdministrative
grossProfit
ebit
nonOperatingIncomeNetOther
operatingIncome
otherOperatingExpenses
interestExpense
taxProvision
interestIncome
netInterestIncome
extraordinaryItems
nonRecurring
otherItems
incomeTaxExpense
totalRevenue
totalOperatingExpenses
costOfRevenue
totalOtherIncomeExpenseNet
discontinuedOperations
netIncomeFromContinuingOps
netIncomeApplicableToCommonShares
preferredStockAndOtherAdjustments


In [24]:
df.head()

,Unternehmen,Branche Detail,Branche aggregiert,Link zu Kennzahlen,stock_ticker,Unnamed: 5,daily_stock
0,1&1 Drillisch,Telekommunikation,Software / Tech,https://www.finanzen.net/aktien/drillisch-aktie,DRI.XETRA,DRI.STU,DRI.XETRA
1,AAR Corp.,Raumfahrt,Aviation,https://www.finanzen.net/aktien/aar-aktie,AIR.US,NaN,AIR.US
2,Activision,Gaming / E-Sport,Software / Tech,https://www.finanzen.net/aktien/activision_bli...,AIY.XETRA,ATVI.US,AIY.XETRA
3,ADIDAS,Sportartikelhersteller,Handel,https://www.finanzen.net/aktien/adidas-aktie,ADS.XETRA,ADDYY.US,ADS.XETRA
4,Adler Real Estate,Immobilien,Immobilien,https://www.finanzen.net/aktien/adler_real_est...,ADL.XETRA,NaN,ADL.XETRA


In [28]:
temp = df['Unternehmen'].where(df['Unternehmen'] == 'RTL Group S.A')

In [38]:
df[df['Unternehmen'] == 'RTL']

,Unternehmen,Branche Detail,Branche aggregiert,Link zu Kennzahlen,stock_ticker,Unnamed: 5,daily_stock
123,RTL,TV / Medien,Medien,https://www.finanzen.net/aktien/rtl-aktie,RRTL.XETRA,NaN,RRTL.XETRA


In [39]:
df.iloc[123:]

,Unternehmen,Branche Detail,Branche aggregiert,Link zu Kennzahlen,stock_ticker,Unnamed: 5,daily_stock
123,RTL,TV / Medien,Medien,https://www.finanzen.net/aktien/rtl-aktie,RRTL.XETRA,NaN,RRTL.XETRA
124,RWE,Energieversorger,Energie / Rohstoffe,https://www.finanzen.net/aktien/rwe-aktie,RWE.XETRA,NaN,RWE.XETRA
125,Safran,Triebwerke,Aviation,https://www.finanzen.net/aktien/safran-aktie,0IU8.LSE,NaN,0IU8.LSE
126,Saint-Gobain,Baustoffe,Energie / Rohstoffe,https://www.finanzen.net/aktien/saint-gobain-a...,COD.LSE,NaN,COD.LSE
127,SAP,Software,Software / Tech,https://www.finanzen.net/aktien/sap-aktie,CODYY.US,NaN,CODYY.US
...,...,...,...,...,...,...,...
151,Walt Disney,TV / Medien,Medien,https://www.finanzen.net/aktien/disney-aktie,DIS.US,WDP.XETRA,DIS.US
152,Wirecard,Zahlungsdienstleister,Finanzwirtschaft,https://www.finanzen.net/aktien/wirecard-aktie,WDI.XETRA,WRCDF.US,WDI.XETRA
153,Xylem,Wasserwirtschaft,Energie / Rohstoffe,https://www.finanzen.net/aktien/xylem-aktie,XYL.US,XY6.XETRA,XYL.US
154,Zalando,E-Commerce,Software / Tech,https://www.finanzen.net/aktien/zalando-aktie,ZAL.XETRA,ZLNDY.US,ZAL.XETRA


In [ ]:
df['preTestScore'].where(df['postTestScore'] > 50)

In [ ]:
for quater in data['Financials']['Cash_Flow']['quarterly']:
    print(pd.DataFrame(data['Financials']['Cash_Flow']['quarterly'][quater], index=[0]).columns)

In [43]:
for year in data['Financials']['Cash_Flow']['yearly']:
    print(pd.DataFrame(data['Financials']['Cash_Flow']['yearly'][year], index=[0]).columns)
    break

Index(['date', 'filing_date', 'currency_symbol', 'investments',
       'changeToLiabilities', 'totalCashflowsFromInvestingActivities',
       'netBorrowings', 'totalCashFromFinancingActivities',
       'changeToOperatingActivities', 'netIncome', 'changeInCash',
       'totalCashFromOperatingActivities', 'depreciation',
       'otherCashflowsFromInvestingActivities', 'dividendsPaid',
       'changeToInventory', 'changeToAccountReceivables',
       'salePurchaseOfStock', 'otherCashflowsFromFinancingActivities',
       'changeToNetincome', 'capitalExpenditures', 'changeReceivables',
       'cashFlowsOtherOperating', 'exchangeRateChanges',
       'cashAndCashEquivalentsChanges'],
      dtype='object')


In [45]:
for quater in data['Financials']['Income_Statement']['quarterly']:
    print(pd.DataFrame(data['Financials']['Income_Statement']['quarterly'][quater], index=[0]).columns)
    break

Index(['date', 'filing_date', 'currency_symbol', 'researchDevelopment',
       'effectOfAccountingCharges', 'incomeBeforeTax', 'minorityInterest',
       'netIncome', 'sellingGeneralAdministrative', 'grossProfit', 'ebit',
       'nonOperatingIncomeNetOther', 'operatingIncome',
       'otherOperatingExpenses', 'interestExpense', 'taxProvision',
       'interestIncome', 'netInterestIncome', 'extraordinaryItems',
       'nonRecurring', 'otherItems', 'incomeTaxExpense', 'totalRevenue',
       'totalOperatingExpenses', 'costOfRevenue', 'totalOtherIncomeExpenseNet',
       'discontinuedOperations', 'netIncomeFromContinuingOps',
       'netIncomeApplicableToCommonShares',
       'preferredStockAndOtherAdjustments'],
      dtype='object')


In [46]:
for year in data['Financials']['Income_Statement']['yearly']:
    print(pd.DataFrame(data['Financials']['Income_Statement']['yearly'][year], index=[0]).columns)
    break

Index(['date', 'filing_date', 'currency_symbol', 'researchDevelopment',
       'effectOfAccountingCharges', 'incomeBeforeTax', 'minorityInterest',
       'netIncome', 'sellingGeneralAdministrative', 'grossProfit', 'ebit',
       'nonOperatingIncomeNetOther', 'operatingIncome',
       'otherOperatingExpenses', 'interestExpense', 'taxProvision',
       'interestIncome', 'netInterestIncome', 'extraordinaryItems',
       'nonRecurring', 'otherItems', 'incomeTaxExpense', 'totalRevenue',
       'totalOperatingExpenses', 'costOfRevenue', 'totalOtherIncomeExpenseNet',
       'discontinuedOperations', 'netIncomeFromContinuingOps',
       'netIncomeApplicableToCommonShares',
       'preferredStockAndOtherAdjustments'],
      dtype='object')
